<a href="https://colab.research.google.com/github/sahinya17/Neural-Network-with-Keras/blob/master/NeuralNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [12]:
%%R
install.packages("keras")
library(tensorflow)
library(keras)

(as ‘lib’ is unspecified)







	‘/tmp/RtmpatxBbR/downloaded_packages’



In [11]:
%%R
library(tidyverse)
theme_set(theme_bw())
library(keras)
library(rsample)

n_epochs <- 2

mnist <- dataset_mnist()

x_train <- mnist$train$x
y_train <- mnist$train$y
x_test <- mnist$test$x
y_test <- mnist$test$y

# reshape
dim(x_train) <- c(nrow(x_train), 784)
dim(x_test) <- c(nrow(x_test), 784)


# rescale
x_train <- x_train / 255
x_test <- x_test / 255

y_train <- to_categorical(y_train, 10)
y_test <- to_categorical(y_test, 10)


# Initialise list
model_list <- list()

# Exercise 1
model_list[["Tutorial"]]$model <- keras_model_sequential() %>%
  layer_dense(units = 256, activation = "relu", input_shape = c(784)) %>%
  layer_dropout(rate = 0.4) %>%
  layer_dense(units = 128, activation = "relu") %>%
  layer_dropout(rate = 0.3) %>%
  layer_dense(units = 10, activation = "softmax") %>%
  compile(loss = "categorical_crossentropy",
          optimizer = optimizer_rmsprop(),
          metrics = c("accuracy"))

model_list[["Tutorial"]]$fit <- model_list[["Tutorial"]]$model %>%
  fit(x_train, y_train, epochs = n_epochs, batch_size = 128,
      validation_split = 0.2)

# Assignment question 1 (a)
model_list[["Dropout"]]$model <- keras_model_sequential() %>%
  layer_dense(units = 128, activation = "relu", input_shape = c(784)) %>%
  layer_dropout(rate = 0.4) %>%
  layer_dense(units = 64, activation = "relu") %>%
  layer_dropout(rate = 0.3) %>%
  layer_dense(units = 10, activation = "softmax") %>%
  compile(loss = "categorical_crossentropy",
          optimizer = optimizer_rmsprop(),
          metrics = c("accuracy"))

model_list[["Dropout"]]$fit <- model_list[["Dropout"]]$model %>%
  fit(x_train, y_train, epochs = n_epochs, batch_size = 128,
      validation_split = 0.2)

# Assignment question 1 (b)
model_list[["L1"]]$model <- keras_model_sequential() %>%
  layer_dense(units = 128, activation = "relu", input_shape = c(784),
              kernel_regularizer = regularizer_l1(1e-4)) %>%
  layer_dense(units = 64, activation = "relu",
              kernel_regularizer = regularizer_l1(1e-4)) %>%
  layer_dense(units = 10, activation = "softmax") %>%
  compile(loss = "categorical_crossentropy",
          optimizer = optimizer_rmsprop(),
          metrics = c("accuracy"))

model_list[["L1"]]$fit <- model_list[["L1"]]$model %>%
  fit(x_train, y_train, epochs = n_epochs, batch_size = 128,
      validation_split = 0.2)

# Assignment question 1 (c)
model_list[["Early stopping"]]$model <- keras_model_sequential() %>%
  layer_dense(units = 128, activation = "relu", input_shape = c(784)) %>%
  layer_dense(units = 64, activation = "relu") %>%
  layer_dense(units = 10, activation = "softmax") %>%
  compile(loss = "categorical_crossentropy",
          optimizer = optimizer_rmsprop(),
          metrics = c("accuracy"))

model_list[["Early stopping"]]$fit <- model_list[["Early stopping"]]$model %>%
  fit(x_train, y_train, epochs = n_epochs, batch_size = 128,
      validation_split = 0.2,
      callbacks = callback_early_stopping(monitor = 'val_loss',
                                          patience = 2))


# Evaluate models
evaluate_df <- function(model) {
  evaluate(model, x_test, y_test, verbose = 0) %>%
    bind_rows()
}

# convert list to data frame for easier processing
model_df <- data_frame(
  name = fct_inorder(names(model_list)), # use factor for plot order
  model = map(model_list, "model"),  # pull out model element
  fit = map(model_list, "fit")  # pull out fit element
) %>%
  mutate(eval = map(model, evaluate_df)) %>%
  unnest(eval)

k_clear_session()

# Plots - alternative commented out below
# walk2(model_df$fit, model_df$name,
#       function(fit, name) {
#         # plot method for keras_training_history object returns ggplot object
#         p <- plot(fit) +
#           ggtitle(name)
#         print(p)
#       })

model_df %>%
  mutate(metrics = map(fit, function(x) { data.frame(x) })) %>%
  select(name, metrics) %>%
  unnest(metrics) %>%
  ggplot(aes(x = epoch, y = value, colour = data)) +
  geom_line() +
  facet_grid(metric~name, scales = "free_y")

model_df %>%
  select(name, loss, accuracy) %>% 
  knitr::kable(caption = "Loss and accuracy for Keras models")

model_df %>%
  select(name, loss, accuracy) %>%
  gather(var, val, -name) %>%
  ggplot(aes(x = name, y = val)) +
  geom_col(colour = "blue", fill = "blue", alpha = 0.3, width = 0) +
  geom_point(colour = "blue") +
  facet_wrap(~var, scales = "free_x") +
  coord_flip()


Error in library(keras) : there is no package called ‘keras’


RInterpreterError: ignored